Rename a bunch of files using Git and renamer.

In [16]:
var execSync = require('child_process').execSync;
try {
    require.resolve('renamer');
} catch (e) {
    execSync('npm install renamer');
}

var renamer = require('renamer');
var files = renamer.expand('C:\\Users\\brian.cullinan\\Documents\\portal\\src\\**\\*.scss');
var results = renamer.replace({files: files.filesAndDirs, find: '.scss', replace: '.less'});
renamer.dryRun(results).list.forEach(r => {
    console.log(r.before);
    // rename with git instead
    execSync('git --work-tree="C:\\Users\\brian.cullinan\\Documents\\portal" --git-dir="C:\\Users\\brian.cullinan\\Documents\\portal\\.git" mv ' + JSON.stringify(r.before) + ' ' + JSON.stringify(r.after));
});

Generate a bunch of spec files.

In [33]:
var execSync = require('child_process').execSync;
try {
    require.resolve('glob');
} catch (e) {
    execSync('npm install glob');
}
var fs = require('fs');
var glob = require('glob');
var path = require('path');

var cwd = 'C:\\Users\\brian.cullinan\\Documents\\portal\\src\\app\\subscriptions';

var spec = `
import {ComponentFixture} from '@angular/core/testing';
const assert = require('assert');
import {} from 'jasmine';
import {MockBackend} from '@angular/http/testing';
import {Router} from '@angular/router';
import {Location} from '@angular/common';
import {By} from '@angular/platform-browser';
import {Core} from '../../../testing/core';
{import}

describe('{title}', function () {
    let fixture: ComponentFixture<{component}>;
    let router: Router;
    let location: Location;
    let xhr: MockBackend;
    let comp: {component};

    beforeEach(() => {
        fixture = Core.createAppWithShared([], [], [{component}]);
        comp = fixture.componentInstance;
        router = fixture.debugElement.injector.get(Router);
        xhr = fixture.debugElement.injector.get(MockBackend);
        location = fixture.debugElement.injector.get(Location);
    });

    it('should load {component} component', (done) => {
        fixture.whenStable().then(() => {
            expect(comp).not.toBeNull();
            done();
        });
    });

});
`;

var files = glob.sync('**/*component.ts', {
    ignore: '**/*.spec.ts',
    cwd: cwd
});

var titles = files.map(f => {
    var name = f.split('.')[0].split('/').pop().replace();
    var ucFirst = name.replace(/(^|\s|-)([a-z])/g , (m,p1,p2) => p1+p2.toUpperCase());
    return ucFirst.replace(/-/ig, '');
})

titles.forEach((t, i) => {
    try {
        var specPath = path.join(cwd, files[i].replace('.ts', '.spec.ts'));
        var fileStr = files[i].split('/').pop().replace('.ts', '');
        var importStr = 'import {' + t + 'Component} from \'./' + fileStr + '\';';
        fs.writeFileSync(specPath, spec
                         .replace(/\{component\}/ig, t + 'Component')
                         .replace(/\{title\}/ig, t)
                         .replace(/\{import\}/ig, importStr));
    } catch (e) {}
});


Replace package.json latest with actual latest version numbers

In [16]:
var fs = require('fs');
var path = require('path');
var cwd = 'C:\\Users\\brian.cullinan\\Documents\\portal\\';
var execSync = require('child_process').execSync;
var packages = execSync('npm list --dev', {cwd: cwd}).toString().split('\n');
var packageJson = JSON.parse(fs.readFileSync(path.join(cwd, 'package.json')));
var deps = Object.keys(packageJson.dependencies);
deps.forEach((d, k) => {
    packages.forEach((p, i) => {
        if(p.indexOf(d + '@') > -1) {
            packageJson.dependencies[d] = '~' + p.split('@').pop().replace(' deduped', '');
        }
    });
});
console.log(packages);
var devDeps = Object.keys(packageJson.devDependencies);
devDeps.forEach((d, k) => {
    packages.forEach((p, i) => {
        if(p.indexOf(d + '@') > -1) {
            packageJson.devDependencies[d] = '~' + p.split('@').pop().replace(' deduped', '');
        }
    });
});
$$.mime({'text/html': '<pre>' + JSON.stringify(packageJson, null, 4) + '</pre>'});


[ 'act-ecommerce@2.0.0 C:\\Users\\brian.cullinan\\Documents\\portal',
  '+-- @angular/animations@4.1.3',
  '+-- @angular/cli@1.1.1',
  '| +-- @ngtools/json-schema@1.1.0',
  '| +-- @ngtools/webpack@1.4.1',
  '| | +-- enhanced-resolve@3.1.0 deduped',
  '| | +-- loader-utils@1.1.0 deduped',
  '| | +-- magic-string@0.19.1',
  '| | | `-- vlq@0.2.2',
  '| | `-- source-map@0.5.6 deduped',
  '| +-- autoprefixer@6.7.7',
  '| | +-- browserslist@1.7.7',
  '| | | +-- caniuse-db@1.0.30000683 deduped',
  '| | | `-- electron-to-chromium@1.3.14 deduped',
  '| | +-- caniuse-db@1.0.30000683',
  '| | +-- normalize-range@0.1.2',
  '| | +-- num2fraction@1.2.2',
  '| | +-- postcss@5.2.17 deduped',
  '| | `-- postcss-value-parser@3.3.0 deduped',
  '| +-- chalk@1.1.3',
  '| | +-- ansi-styles@2.2.1',
  '| | +-- escape-string-regexp@1.0.5',
  '| | +-- has-ansi@2.0.0',
  '| | | `-- ansi-regex@2.1.1',
  '| | +-- strip-ansi@3.0.1',
  '| | | `-- ansi-regex@2.1.1 deduped',
  '| | `-- supports-color@2.0.0',
  '| +-- 

{
 "name": "act-ecommerce",
 "version": "2.0.0",
 "description": "Swiftpage Ecommerce Portal",
 "main": "index.ts",
 "scripts": {
 "start": "ng serve --host 0.0.0.0 --port 9090",
 "build": "ng build --prod --aot",
 "lint": "tslint ./src/**/*.ts",
 "test": "karma start --browsers Chrome"
 },
 "repository": {
 "type": "git",
 "url": "git+https://github.com/Swiftpage/act.ecommerce.git"
 },
 "author": "Swiftpage, LLC",
 "license": "UNLICENSED",
 "bugs": {
 "url": "https://github.com/Swiftpage/act.ecommerce/issues"
 },
 "homepage": "https://github.com/Swiftpage/act.ecommerce#readme",
 "engines": {
 "node": ">= 4.2.1",
 "npm": ">= 3"
 },
 "private": true,
 "dependencies": {
 "@angular/animations": "~4.1.3",
 "@angular/common": "~4.1.3",
 "@angular/compiler": "~4.1.3",
 "@angular/core": "~4.1.3",
 "@angular/forms": "~4.1.3",
 "@angular/http": "~4.1.3",
 "@angular/material": "~2.0.0-beta.6",
 "@angular/platform-browser": "~4.1.3",
 "@angular/platform-browser-dynamic": "~4.1.3",
 "@angular/platform-server": "~4.1.3",
 "@angular/router": "~4.1.3",
 "@ngx-translate/core": "~7.0.0",
 "@ngx-translate/http-loader": "~0.1.0",
 "angular2-jwt": "~0.2.3",
 "babel-polyfill": "~6.23.0",
 "copy-webpack-plugin": "~4.0.1",
 "core-js": "~2.4.1 deduped",
 "jquery": "~3.2.1",
 "rxjs": "~5.4.0",
 "zone.js": "~0.8.12"
 },
 "devDependencies": {
 "@angular/cli": "~1.1.1",
 "@angular/compiler-cli": "~4.1.3",
 "@types/jasmine": "~2.5.51",
 "@types/node": "~6.0.78",
 "awesome-typescript-loader": "~3.1.3",
 "angular2-router-loader": "~0.3.5",
 "angular2-template-loader": "~0.6.2",
 "babel-core": "~6.25.0 deduped",
 "babel-plugin-istanbul": "~4.1.4",
 "babel-preset-env": "~1.5.2",
 "babel-preset-es2015": "~6.24.1",
 "codelyzer": "~3.0.1",
 "css-loader": "~0.28.4",
 "css-to-string-loader": "~0.1.3",
 "html-loader": "~0.4.5",
 "istanbul-instrumenter-loader": "~2.0.0",
 "jasmine-core": "~2.6.3 deduped",
 "jasmine-spec-reporter": "~4.1.0",
 "karma": "~1.7.0",
 "karma-chrome-launcher": "~2.1.1",
 "karma-cli": "~1.0.1",
 "karma-coverage": "~1.1.1",
 "karma-coverage-istanbul-reporter": "~1.3.0",
 "karma-jasmine": "~1.1.0 deduped",
 "karma-jasmine-html-reporter": "~0.2.2",
 "karma-sourcemap-loader": "~0.3.7",
 "karma-spec-reporter": "~0.0.31",
 "karma-webpack": "~2.0.3",
 "less-loader": "~4.0.4",
 "protractor": "~5.1.2",
 "sass-loader": "~6.0.5",
 "ts-loader": "~2.1.0",
 "ts-node": "~3.0.6",
 "tslint": "~5.4.3",
 "typescript": "~2.3.4",
 "copy-webpack-plugin": "~4.0.1"
 },
 "keywords": []
}

Find unused/misplaced translation strings

In [116]:
var glob = require('glob');
var path = require('path');
var fs = require('fs');
var translateRegex = (/[\{"]([^><"]*?)\s*\|\s*translate\s*["\}]/ig);
var cwd = 'C:\\Users\\brian.cullinan\\Documents\\portal\\src\\';
var files = glob.sync('**/*component.html', {
    ignore: '**/temp-demo-pages/**',
    cwd: cwd
});
var translations = files.map((f, i) => {
    var html = fs.readFileSync(path.join(cwd, f)).toString();
    let r;
    var translateStrings = [];
    while((r = translateRegex.exec(html)) !== null) {
        translateStrings[translateStrings.length] = r[1].trim().replace(/[\'{}\s]/ig, '');
    }
    var desiredKey = f
                .replace('app/', '')
                .replace(/\//ig, '.')
                .replace('.component.html', '')
                .replace(/-/ig, '').split('.');
    desiredKey.pop();
    desiredKey = desiredKey.join('.').toUpperCase();
    var result = {};
    result[desiredKey] = translateStrings;
    return result;
});

var translationKeys = Object.assign.apply({}, translations);

function findMisplaced(obj, parentKey)
{
    for (var k in obj)
    {
        if (typeof obj[k] == "object" && obj[k] !== null) {
            var newParent = typeof parentKey !== 'undefined' ? (parentKey + '.' + k) : k;
            findMisplaced(obj[k], newParent);
        } else if(typeof obj[k] == "string") {
            if(obj[k].substr(0, parentKey.length) != parentKey) {
                console.log('Misplaced key: ' + obj[k] + ' in ' + parentKey);
            }
        }
    }
}

findMisplaced(translationKeys);

// get a long list of existing keys from en.js
var enJson = JSON.parse(fs.readFileSync(path.join(cwd, 'assets', 'i18n', 'en.json')).toString());


function flattenAllKeys(obj, parentKey)
{
    var result = [];
    for (var k in obj)
    {
        if (typeof obj[k] == "object" && obj[k] !== null) {
            var newParent = typeof parentKey !== 'undefined' ? (parentKey + '.' + k) : k;
            result = result.concat(flattenAllKeys(obj[k], newParent));
        } else if(typeof obj[k] == "string") {
            result[result.length] = parentKey + '.' + k;
        }
    }
    return result;
}

var allENKeys = flattenAllKeys(enJson);

// find unused keys
function getUnused() {
    allENKeys.forEach(k => {
        var parentKey = k.split('.');
        parentKey.pop();
        parentKey = parentKey.join('.');
        if(typeof translationKeys[parentKey] === 'undefined' ||
            translationKeys[parentKey].indexOf(k) === -1) {
            console.log('Unused key: ' + k);
        }
    });
}

function getMissing() {
    for(var k in translationKeys) {
        if(typeof translationKeys[k] !== 'undefined') {
            translationKeys[k].forEach(i => {
                if(allENKeys.indexOf(i) === -1) {
                    console.log('Missing key: ' + i);
                }
            });
        }
    }
}

getMissing();
getUnused();


// find strings of text in html files
var htmlWordRegex = (/>[^><]*?(\b[^><]*\b)+[^><]*?</ig)
var attrWordRegex = (/(placeholder|title|alt)\s*=\s*["'](.*)["']/ig);
var needTranslations = files.map((f, i) => {
    var html = fs.readFileSync(path.join(cwd, f)).toString();
    let r;
    var needTranslations = [];
    while((r = htmlWordRegex.exec(html)) !== null) {
        if(r[1].trim() !== '' && r[1].match(/\|\s*translate/ig) === null) {
            needTranslations[needTranslations.length] = r[1].trim();
        }
    }
    while((r = attrWordRegex.exec(html)) !== null) {
        if(r[2].match(/\|\s*translate/ig) === null) {
            needTranslations[needTranslations.length] = r[2].trim();
        }
    }
    return {file: f, texts: needTranslations};
}).filter(t => t.texts.length > 0);

needTranslations.forEach(t => {
    console.log('Needs translating: ' + t.texts + ' in ' + t.file);
});

Misplaced key: +(language|async) in LAYOUT.LANGUAGE
Misplaced key: +lang in LAYOUT.LANGUAGE
Missing key: AUTH.LOGIN.ERRORINCORRECT
Missing key: +(language|async)
Missing key: +lang
Unused key: LAYOUT.LANGUAGE.en
Unused key: LAYOUT.LANGUAGE.fr
Unused key: LAYOUT.LANGUAGE.tr
